## libs

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re

import string
from bs4 import BeautifulSoup
# import requests
# from collections import Counter
# from num2words import num2words
from wordcloud import WordCloud, ImageColorGenerator
from nltk.tokenize import word_tokenize
from tqdm.auto import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
# import spacy
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

import nltk
import warnings
warnings.filterwarnings("ignore")

# from nltk.corpus import stopwords
from google.colab import drive
drive.mount('/content/drive')

pd.set_option('max_colwidth', 99999)

[nltk_data] Downloading package wordnet to /root/nltk_data...


Mounted at /content/drive


In [2]:
import tensorflow as tf
import tensorflow_hub as hub

In [3]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel,TFBertLMHeadModel
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
# from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
# from tensorflow.keras.preprocessing import image as keras_image
# from tensorflow.keras.models import Model
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.utils import to_categorical

# Data Collection

In [4]:
# import os
# import pandas as pd

# # Define the path to your main folder
# main_folder = '/content/drive/MyDrive/ML Caption Craft/Test_Data/'

# # Function to extract captions from text files
# def extract_caption(folder_path):
#     captions = {}
#     for file_name in os.listdir(folder_path):
#         if file_name.endswith('.txt'):
#             with open(os.path.join(folder_path, file_name), 'r') as file:
#                 captions[file_name] = file.read().strip()
#     return captions

# # Initialize empty lists for columns
# img_p = []
# base_model_caption = []
# finetuned_model_caption = []
# groundtruth_caption = []

# # Iterate through subfolders
# for subfolder in tqdm(os.listdir(main_folder)):
#     # print(subfolder)
#     subfolder_path = os.path.join(main_folder, subfolder)
#     if os.path.isdir(subfolder_path):
#         # Get image name
#         # print(subfolder_path)
#         img_name = os.listdir(subfolder_path)[0]
#         img_p.append(img_name)

#         # Extract captions
#         captions = extract_caption(subfolder_path)
#         # print(captions)
#         # Append captions to respective columns
#         base_model_caption.append(captions.get('base_model_caption.txt'))
#         finetuned_model_caption.append(captions.get('finetuned_model_caption.txt'))
#         groundtruth_caption.append(captions.get('groundtruth_caption.txt'))

# # Create a DataFrame with the collected data
# data = {
#     'img_p': img_p,
#     'base_model_caption': base_model_caption,
#     'finetuned_model_caption': finetuned_model_caption,
#     'groundtruth_caption': groundtruth_caption
# }

# # Create a pandas DataFrame
# df = pd.DataFrame(data)

# # Display the DataFrame
# # print(df)

# # Save DataFrame to a CSV file
# df.to_csv('captions_dataset.csv', index=False)

In [5]:
df = pd.read_csv('/content/drive/MyDrive/ML Caption Craft/ALL_PREDICTIONS_1200.csv', lineterminator='\n')

In [6]:
df.head(3)

,Unnamed: 0,img_p,caption,preprocessed_captions,base_vit_gpt_caption,base_coca_caption,generated_caption
0,0,1860421016364639460.jpg,"Am I the only one who’s still dressing up like summer? Well, I will try to enjoy these last couple hot days that we have here so yeah I don’t mind it at all☀️☀️\nT-shirt is from my Etsy store, click the link in my bio to shop.\nI’ve linked the rest of the outfit @liketoknow.it app by following me @yourfrugalista.\n #liketkit #LTKunder100 #LTKshoecrush #LTKunder50 http://liketk.it/2xcGs •\n•\n•\n#labordaylook #labordaystyle #ootdbostonian #denimskirts #graphicteeshirt #slogantees #etsysellers #etsymakers #chicnova #bostonfashionblogger #bostoninfluencers #womanentrepreneur #boaterhat #linenblazer #womanbussinessowner #tjmaxxfinds #targetstyle",am i the only one who is still dressing up like summer well i will try to enjoy these last couple hot days that we have here so yeah i do not mind it at all t shirt is from my etsy store click the link in my bio to shop i have linked the rest of the outfit it app by following me,a woman wearing a hat standing next to a tree,a woman standing next to a tree in a park .,i am so excited to finally be able to wear this dress it is so soft and cute i love how it is made i am wearing it on top of my dress and it is on the side of the tree i have been wearing it for a few days now and i am loving it shop your screenshot of this pic with the liketoknow it app or click the link in my bio to shop this look and others it also has a discount code for off this dress is from and is currently on sale you can shop my looks by following me on
1,1,1862029415539853473.jpg,"End of summer dress, it still feels like summer here so yes I haven’t featured any fall outfits on purpose, I would like to savor this moment as long as possible.\nFollow me @liketoknow.it free app to shop all my looks, search me under @yourfrugalista. #liketkit #LTKitbag #LTKunder100 #LTKunder50 #LTKshoecrush http://liketk.it/2xeva •\n•\n•\n#endofsummerseason #summerdressing #offtheshoulderdress #bostonfashionblogger #ootdbostonian #asianinfluencers #ootdfashionlook #ootdfashionindo #persueyourpassion #newenglandstyle #tjmaxxfinds #zarashoes #ebayfashion",end of summer dress it still feels like summer here so yes i have not featured any fall outfits on purpose i would like to savor this moment as long as possible follow me it free app to shop all my looks search me under,a woman standing in front of a brick wall wearing a pink dress,a woman standing on the side of a road wearing a dress .,i am so excited to finally be able to share my story with you all i am so happy to have you all and thank you so much for taking the time to talk to me i hope you all are having a great time it is been a long time since i have been able to post on my blog but i am excited to share with you some of my favorite things i have seen in the past i have always been a big fan of and i am really excited to see what she is going to bring to the table she is a beautiful woman with a beautiful face and a beautiful
2,2,1867147614090586401.jpg,"It’s actually cold enough to wear my camo jacket today and of course I pair it with my tee from my Etsy store. The sale is still going on guys, so check it out and don’t miss out!\nYou can shop all my looks @liketoknow.it free app, follow me under #yourfrugalista\n #liketkit #LTKunder50 #LTKstyletip #LTKunder100 #LTKxMFW http://liketk.it/2xk4P •\n•\n•\n#etsysales #etsymakers #graphicteeshirt #slogantshirt #supportsmallbusiness #camojackets #fallvibes #fallfashionista #ootdbostonian #stylecommunity #fashionlookbook #bostonfashionista #styleinthecity #tjmaxxfinds #primark",it is actually cold enough to wear my camo jacket today and of course i pair it with my tee from my etsy store the sale is still going on guys so check it out and do not miss out you can shop all my looks it free app follow me under,a woman standing next to a fence with a hat on,a woman standing on the sidewalk wearing a hat .,i 

In [7]:
df.shape

(1237, 7)

# Embedding Generation

In [13]:
# Tokenizer for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [14]:
# bert_model_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
bert_layer = hub.KerasLayer(bert_model, trainable=False)

In [15]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [9]:
likable_data = pd.read_csv('/content/drive/MyDrive/ML Caption Craft/clean_data.csv', lineterminator='\n')

In [16]:
# Tokenizer for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

captions = likable_data['caption'].tolist()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

NameError: ignored

In [ ]:
len(captions)

In [ ]:
bert_embeddings = []
batch_size = 50
num_samples = len(captions)
num_batches = (num_samples + batch_size - 1) // batch_size

max_seq_length = 500

for i in tqdm(range(num_batches)):
    start = i * batch_size
    end = min((i + 1) * batch_size, num_samples)
    batch_captions = captions[start:end]

    tokenized_captions = tokenizer(batch_captions, padding='max_length', truncation=True, max_length=max_seq_length, return_tensors="tf")
    input_word_ids = tokenized_captions['input_ids']
    input_mask = tokenized_captions['attention_mask']
    input_type_ids = tokenized_captions['token_type_ids']

    # Obtain BERT embeddings for the batch on GPU
    with tf.device('/GPU:0'):
        batch_bert_outputs = bert_model([input_word_ids, input_mask, input_type_ids])

    # For BERT models, the output may have different keys like 'last_hidden_state', 'pooler_output', etc.
    # Let's assume you want to use 'pooler_output'
    pooled_output = batch_bert_outputs.pooler_output

    # Append the embeddings to the list
    bert_embeddings.extend(pooled_output.numpy())

# Convert the embeddings list to a numpy array
bert_embeddings = np.array(bert_embeddings)

In [ ]:
np.save('/content/drive/MyDrive/ML Caption Craft/clean_embeddings.npy', bert_embeddings)

# Embedding Similarity check metric

In [13]:
# Tokenizer for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [8]:
bert_embeddings = np.load('/content/drive/MyDrive/ML Caption Craft/clean_embeddings.npy')

In [9]:
bert_embeddings.shape

(17237, 768)

In [10]:
def get_bert_embedding(sentences):
  pred_captions = sentences
  pred_embeddings = []
  batch_size = 100
  num_samples = len(pred_captions)
  num_batches = (num_samples + batch_size - 1) // batch_size

  max_seq_length = 500

  for i in tqdm(range(num_batches)):
      start = i * batch_size
      end = min((i + 1) * batch_size, num_samples)
      batch_captions = pred_captions[start:end]

      tokenized_captions = tokenizer(batch_captions, padding='max_length', truncation=True, max_length=max_seq_length, return_tensors="tf")
      input_word_ids = tokenized_captions['input_ids']
      input_mask = tokenized_captions['attention_mask']
      input_type_ids = tokenized_captions['token_type_ids']

      # Obtain BERT embeddings for the batch on GPU
      with tf.device('/GPU:0'):
          batch_bert_outputs = bert_model([input_word_ids, input_mask, input_type_ids])

      # For BERT models, the output may have different keys like 'last_hidden_state', 'pooler_output', etc.
      # Let's assume you want to use 'pooler_output'
      pooled_output = batch_bert_outputs.pooler_output

      # Append the embeddings to the list
      pred_embeddings.extend(pooled_output.numpy())
      return np.array(pred_embeddings)

In [11]:
df.head(3)

,Unnamed: 0,img_p,caption,preprocessed_captions,base_vit_gpt_caption,base_coca_caption,generated_caption
0,0,1860421016364639460.jpg,"Am I the only one who’s still dressing up like summer? Well, I will try to enjoy these last couple hot days that we have here so yeah I don’t mind it at all☀️☀️\nT-shirt is from my Etsy store, click the link in my bio to shop.\nI’ve linked the rest of the outfit @liketoknow.it app by following me @yourfrugalista.\n #liketkit #LTKunder100 #LTKshoecrush #LTKunder50 http://liketk.it/2xcGs •\n•\n•\n#labordaylook #labordaystyle #ootdbostonian #denimskirts #graphicteeshirt #slogantees #etsysellers #etsymakers #chicnova #bostonfashionblogger #bostoninfluencers #womanentrepreneur #boaterhat #linenblazer #womanbussinessowner #tjmaxxfinds #targetstyle",am i the only one who is still dressing up like summer well i will try to enjoy these last couple hot days that we have here so yeah i do not mind it at all t shirt is from my etsy store click the link in my bio to shop i have linked the rest of the outfit it app by following me,a woman wearing a hat standing next to a tree,a woman standing next to a tree in a park .,i am so excited to finally be able to wear this dress it is so soft and cute i love how it is made i am wearing it on top of my dress and it is on the side of the tree i have been wearing it for a few days now and i am loving it shop your screenshot of this pic with the liketoknow it app or click the link in my bio to shop this look and others it also has a discount code for off this dress is from and is currently on sale you can shop my looks by following me on
1,1,1862029415539853473.jpg,"End of summer dress, it still feels like summer here so yes I haven’t featured any fall outfits on purpose, I would like to savor this moment as long as possible.\nFollow me @liketoknow.it free app to shop all my looks, search me under @yourfrugalista. #liketkit #LTKitbag #LTKunder100 #LTKunder50 #LTKshoecrush http://liketk.it/2xeva •\n•\n•\n#endofsummerseason #summerdressing #offtheshoulderdress #bostonfashionblogger #ootdbostonian #asianinfluencers #ootdfashionlook #ootdfashionindo #persueyourpassion #newenglandstyle #tjmaxxfinds #zarashoes #ebayfashion",end of summer dress it still feels like summer here so yes i have not featured any fall outfits on purpose i would like to savor this moment as long as possible follow me it free app to shop all my looks search me under,a woman standing in front of a brick wall wearing a pink dress,a woman standing on the side of a road wearing a dress .,i am so excited to finally be able to share my story with you all i am so happy to have you all and thank you so much for taking the time to talk to me i hope you all are having a great time it is been a long time since i have been able to post on my blog but i am excited to share with you some of my favorite things i have seen in the past i have always been a big fan of and i am really excited to see what she is going to bring to the table she is a beautiful woman with a beautiful face and a beautiful
2,2,1867147614090586401.jpg,"It’s actually cold enough to wear my camo jacket today and of course I pair it with my tee from my Etsy store. The sale is still going on guys, so check it out and don’t miss out!\nYou can shop all my looks @liketoknow.it free app, follow me under #yourfrugalista\n #liketkit #LTKunder50 #LTKstyletip #LTKunder100 #LTKxMFW http://liketk.it/2xk4P •\n•\n•\n#etsysales #etsymakers #graphicteeshirt #slogantshirt #supportsmallbusiness #camojackets #fallvibes #fallfashionista #ootdbostonian #stylecommunity #fashionlookbook #bostonfashionista #styleinthecity #tjmaxxfinds #primark",it is actually cold enough to wear my camo jacket today and of course i pair it with my tee from my etsy store the sale is still going on guys so check it out and do not miss out you can shop all my looks it free app follow me under,a woman standing next to a fence with a hat on,a woman standing on the sidewalk wearing a hat .,i 

In [25]:
# base_vit_gpt_caption	base_coca_caption	generated_caption
pred_captions = df['generated_caption'].tolist()


pred_embeddings = get_bert_embedding(pred_captions)

  0%|          | 0/13 [00:00<?, ?it/s]

In [26]:
len(pred_captions)

1237

In [27]:
pred_embeddings.shape

(100, 768)

## checking similarity

In [28]:
similarities = cosine_similarity(pred_embeddings, bert_embeddings)

# Average similarity as a quantitative metric
average_similarity = np.mean(similarities)

print(f"Average cosine similarity between the new sentence and existing embeddings: {average_similarity}")

Average cosine similarity between the new sentence and existing embeddings: 0.9032967686653137


In [ ]:
base_vit_gpt_caption = 0.8727778196334839
base_coca_caption = 0.8757905960083008
generated_caption = 0.9032967686653137

In [14]:
embedding_similarity_results = pd.DataFrame(columns=['Model','Embedding Similarity Score'])
columns =['base_vit_gpt_caption',	'base_coca_caption'	, 'generated_caption']
models = ['Baseline(ViT-GPT2)','Baseline(CoCa)','CaptionWizard']
for col in range(len(columns)):
  col_name = columns[col]
  pred_captions = df[col_name].tolist()

  pred_embeddings = get_bert_embedding(pred_captions)

  similarities = cosine_similarity(pred_embeddings, bert_embeddings)

  # Average similarity as a quantitative metric
  average_similarity = np.mean(similarities)
  row = {'Model': [models[col]], 'Embedding Similarity Score':[average_similarity] }
  df2 = pd.DataFrame(row)
  embedding_similarity_results = pd.concat([embedding_similarity_results, df2], ignore_index = True)

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [15]:
embedding_similarity_results

,Model,Embedding Similarity Score
0,Baseline(ViT-GPT2),0.872778
1,Baseline(CoCa),0.875791
2,CaptionWizard,0.910315
